# Pipelining agents

In [21]:
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import SequentialChain
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
import os

In [ ]:
# ──────────────────────────────────────────────────────────────────────────────
# 1. Configuración de la API Key de OpenAI
# ──────────────────────────────────────────────────────────────────────────────
# Asegúrate de haber exportado tu clave en OPENAI_KEY:
#   export OPENAI_KEY="tu_api_key_de_openai"
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_KEY")

# ──────────────────────────────────────────────────────────────────────────────
# 2. Inicializar el modelo de lenguaje (ChatOpenAI)
# ──────────────────────────────────────────────────────────────────────────────
# Usamos 'gpt-3.5-turbo' con temperatura moderada (0.7) para equilibrio entre
# coherencia y creatividad.  [oai_citation:4‡LangChain Python API](https://api.python.langchain.com/en/latest/chains/langchain.chains.sequential.SequentialChain.html?utm_source=chatgpt.com) [oai_citation:5‡Medium](https://medium.com/data-and-beyond/chains-in-langchain-part-1-040624795f91?utm_source=chatgpt.com)
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.7)

# ──────────────────────────────────────────────────────────────────────────────
# 3. Definir los prompts y LLMChain para cada paso del flujo
# ──────────────────────────────────────────────────────────────────────────────

# 3.1. Paso 1: Resumir el texto original
# Input: {"text": "<texto completo>"}
# Output: {"summary": "<resumen>"}
summarize_prompt = PromptTemplate(
    input_variables=["text"],
    template=(
        "Resume el siguiente texto en un párrafo breve y conciso:\n\n"
        "{text}\n\n"
        "Resumen:"
    )
)
# summarize_chain = LLMChain(
#     llm=llm,
#     prompt=summarize_prompt,
#     output_key="summary"
# )  # El output_key define el nombre de la variable de salida para este paso  [oai_citation:6‡LangChain Python API](https://api.python.langchain.com/en/latest/chains/langchain.chains.sequential.SequentialChain.html?utm_source=chatgpt.com) [oai_citation:7‡Medium](https://medium.com/towards-agi/how-to-invoke-langchain-chains-step-by-step-examples-ad15abd7e784?utm_source=chatgpt.com)
summarize_chain = RunnableParallel(
    summary=(summarize_prompt | llm | StrOutputParser())
)


In [23]:
# 3.2. Paso 2: Traducir el resumen al inglés
# Input: {"summary": "<resumen>"}
# Output: {"translation": "<traducción_al_inglés>"}
translate_prompt = PromptTemplate(
    input_variables=["summary"],
    template=(
        "Traduce al inglés el siguiente resumen, cuidando la fidelidad al significado:\n\n"
        "{summary}\n\n"
        "Traducción:"
    )
)
translate_chain = RunnableParallel(
    translation=(translate_prompt | llm | StrOutputParser())
)

In [24]:
# 3.3. Paso 3: Explicar la traducción a nivel de niño de 5 años
# Input: {"translation": "<texto_en_inglés>"}
# Output: {"explanation": "<explicación_sencilla>"}
explain_prompt = PromptTemplate(
    input_variables=["translation"],
    template=(
        "Explica este texto en inglés como si se lo contaras a un niño de 5 años, "
        "usando ejemplos simples y lenguaje muy claro:\n\n"
        "{translation}\n\n"
        "Explicación:"
    )
)

explain_chain = RunnableParallel(
    explanation=(explain_prompt | llm | StrOutputParser())
)

In [26]:
# ──────────────────────────────────────────────────────────────────────────────
# 4. Combinar todos los pasos en un SequentialChain
# ──────────────────────────────────────────────────────────────────────────────
# Aquí usamos RunnableParallel para ejecutar los pasos en paralelo
overall_chain = (
    RunnablePassthrough.assign(
        summarize = summarize_chain  # Takes 'text' from input, adds 'summarize' key with its string output
    )
    | RunnablePassthrough.assign(
        translate = lambda x: translate_chain.invoke({"summary": x["summarize"]}) # Takes 'summarize' from dict, adds 'translate' key
    )
    | RunnablePassthrough.assign(
        explain = lambda x: explain_chain.invoke({"translation": x["translate"]}) # Takes 'translate' from dict, adds 'explain' key
    )
)
# ──────────────────────────────────────────────────────────────────────────────
# 5. Ejecutar el flujo con un ejemplo de texto
# ──────────────────────────────────────────────────────────────────────────────
if __name__ == "__main__":
    # Ejemplo de texto a procesar
    input_text = (
        "La inteligencia artificial está revolucionando el mundo. "
        "Desde asistentes virtuales hasta diagnósticos médicos, "
        "su impacto es profundo y transformador."
    )

    # Ejecutar el flujo
    result = overall_chain.invoke({"text": input_text})

    # Mostrar los resultados
    print("Resumen:", result)
    print("Resumen:", result["summarize"])
    print("Traducción:", result["translate"])
    print("Explicación:", result["explain"])

Resumen: {'text': 'La inteligencia artificial está revolucionando el mundo. Desde asistentes virtuales hasta diagnósticos médicos, su impacto es profundo y transformador.', 'summarize': {'summary': 'La inteligencia artificial está teniendo un impacto significativo en diversos ámbitos, desde la creación de asistentes virtuales hasta diagnósticos médicos, revolucionando el mundo y transformando la manera en que interactuamos con la tecnología y abordamos problemas de salud.'}, 'translate': {'translation': 'The artificial intelligence is having a significant impact in various fields, from the creation of virtual assistants to medical diagnoses, revolutionizing the world and transforming the way we interact with technology and address health problems.'}, 'explain': {'explanation': "Imagine that there are really smart robots that can do lots of cool things. They can help us talk to our computers, like when we ask Siri a question. They can also help doctors figure out what's wrong with peopl